In [14]:
import os
import json
from datetime import datetime
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization

# Limpiar claves previas si existen se eliminan archivos de claves antiguas para empezar desde cero y evitar conflictos
if os.path.exists("clave_privada.pem"):
    os.remove("clave_privada.pem")
if os.path.exists("clave_publica.pem"):
    os.remove("clave_publica.pem")

# ==== GENERAR CLAVES ====
def generar_claves_asimetricas():
    clave_privada = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    clave_publica = clave_privada.public_key()

    with open("clave_privada.pem", "wb") as f:
        f.write(clave_privada.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))

    with open("clave_publica.pem", "wb") as f:
        f.write(clave_publica.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))  #Estas claves son esenciales para firmar (privada) y verificar (publica)

# ==== CARGAR CLAVES ====
def cargar_clave_privada():
    with open("clave_privada.pem", "rb") as f:
        return serialization.load_pem_private_key(f.read(), password=None)

def cargar_clave_publica():
    with open("clave_publica.pem", "rb") as f:
        return serialization.load_pem_public_key(f.read())
        #permiten reutilizar las claves almacenadas previamente.
# ==== DOCUMENTO ====
def crear_documento(autor, titulo, contenido, archivo="documento.json"):
    doc = {
        "autor": autor,
        "titulo": titulo,
        "contenido": contenido,
        "fecha_creacion": datetime.now().isoformat()
    }
    with open(archivo, "w") as f:
        json.dump(doc, f, indent=4)
    return archivo

# ==== FIRMA ====
def firmar_documento(ruta_documento, ruta_firma="firma.bin"):
    private_key = cargar_clave_privada()
    with open(ruta_documento, "r") as f:
        mensaje = json.dumps(json.load(f)).encode()
    firma = private_key.sign(
        mensaje,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256()
    )
    with open(ruta_firma, "wb") as f:
        f.write(firma)
    return ruta_firma

# ==== VERIFICACIÓN ====
def verificar_firma(ruta_documento, ruta_firma):
    public_key = cargar_clave_publica()
    with open(ruta_documento, "r") as f:
        mensaje = json.dumps(json.load(f)).encode()
    with open(ruta_firma, "rb") as f:
        firma = f.read()
    try:
        public_key.verify(
            firma,
            mensaje,
            padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            hashes.SHA256()
        )
        print("☑️ La firma es válida.")
        return True
    except Exception as e:
        print("❌ La firma NO es válida.")
        return False

# === CASO 1: Documento Modificado ===
print("🔹 CASO 1: Documento Modificado")
generar_claves_asimetricas()
crear_documento("Juan Pérez", "Informe", "Contenido seguro", "doc1.json")
firmar_documento("doc1.json", "firma1.bin")

# Simular modificación
with open("doc1.json", "r") as f:
    doc = json.load(f)
doc["contenido"] = "Contenido alterado"
with open("doc1_modificado.json", "w") as f:
    json.dump(doc, f, indent=4)

verificar_firma("doc1_modificado.json", "firma1.bin")
print()

# === CASO 2: Firma no corresponde ===
print("🔹 CASO 2: Firma no corresponde")
generar_claves_asimetricas()
crear_documento("Ana", "Doc A", "Texto A", "docA.json")
crear_documento("Carlos", "Doc B", "Texto B", "docB.json")
firmar_documento("docA.json", "firmaA.bin")
verificar_firma("docB.json", "firmaA.bin")
print()

# === CASO 3: Todo Correcto ===
print("🔹 CASO 3: Todo Correcto")
generar_claves_asimetricas()
crear_documento("María", "Auditoría", "Informe legítimo", "doc_ok.json")
firmar_documento("doc_ok.json", "firma_ok.bin")
verificar_firma("doc_ok.json", "firma_ok.bin")

🔹 CASO 1: Documento Modificado
❌ La firma NO es válida.

🔹 CASO 2: Firma no corresponde
❌ La firma NO es válida.

🔹 CASO 3: Todo Correcto
☑️ La firma es válida.


True